In [213]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [214]:
X, y = make_classification(
    n_samples=100, # Number of samples to generate.
    n_features=2, # Total number of features (or variables) for each sample.
    n_informative=1, # Out of the 2 features, only 1 feature will be genuinely informative for the classification task. The informative features are the ones that contribute to distinguishing between classes.
    n_redundant=0, # There are no redundant features. Redundant features are combinations of informative features that are used to create new features but do not provide additional information.
    n_classes=2, # The dataset will have 2 classes. This is a binary classification problem.
    n_clusters_per_class=1, # Each class will be represented by a single cluster. This means that the data points for each class will be grouped into one distinct cluster.
    random_state=23, # seed 
    hypercube=False, # If False, the data is not generated from a hypercube but rather from a Gaussian distribution.
    class_sep=10 # Separation between classes.
)
df = pd.DataFrame(X)
df.rename(columns={0: 'X1', 1: 'X2'}, inplace=True)
df['y'] = y
df.head()

,X1,X2,y
0,-4.473951,-1.436281,0
1,-3.965235,0.515680,0
2,-3.904806,0.429005,0
3,-3.939984,0.020888,0
4,7.374444,0.159692,1


In [215]:
fig = px.scatter(
    df, 
    x='X1', 
    y='X2', 
    color='y',
    labels={'Feature1': 'X1', 'Feature2': 'X2', 'Category': 'Category'}
)
fig.show()

In [216]:
class perceptraon:
    def __init__(self, lr, epochs) -> None:
        self.coef_ = None
        self.intercept_ = None
        self.epochs = epochs
        self.learning_rate = lr
    
    def fit(self, X_train, y_train):
        X_train = np.array(X_train)
        y_train = np.array(y_train).ravel()

        X_train = np.insert(X_train, 0, 1, axis = 1)

        beta = np.ones(X_train.shape[1])

        for i in range(self.epochs):
            idx = np.random.randint(0, X_train.shape[0]) # Data Point
            y_hat = 1 if np.dot(X_train[idx], beta) >= 0 else 0 # Result -> 1 or 0
            beta = beta + self.learning_rate * (y_train[idx] - y_hat)*X_train[idx]
        
        self.coef_ = beta[1:]
        self.intercept_ = beta[0]

In [217]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [218]:
model = perceptraon(0.01, 1000)
model.fit(X_train, y_train)

In [219]:
print(model.coef_, model.intercept_)

[1. 1.] 1.0


In [220]:
m = -model.coef_[0]/model.coef_[1]
b = -model.intercept_/model.coef_[1]

In [221]:
category0 = df[df['y'] == 0]
category1 = df[df['y'] == 1]

fig = go.Figure()
fig.add_trace(go.Scatter(x = category0['X1'], y = category0['X2'], name = '1st category', mode = 'markers', marker=dict(
        color='red',
    )))
fig.add_trace(go.Scatter(x = category1['X1'], y = category1['X2'], name = '2st category', mode = 'markers', marker=dict(
        color='blue',
    )))
fig.add_trace(go.Scatter(x = df['X1'], y = m*df['X1'] + b, mode = 'lines', name='Line of Classification'))
fig.update_layout(width= 900, height= 500)

fig.update_yaxes(range=[-2.5, 2.7])

fig.show()

In [222]:
model = LogisticRegression(penalty=None,solver='sag')
model.fit(X_train, y_train)

c:\Users\shrir\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



LogisticRegression(penalty=None, solver='sag')

In [223]:
model.intercept_

array([0.11217206])

In [224]:
model_m = -model.coef_[0, 0]/model.coef_[0, 1]
model_b = -model.intercept_[0]/model.coef_[0, 1]

In [225]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = category0['X1'], y = category0['X2'], name = '1st category', mode = 'markers', marker=dict(
        color='red',
    )))
fig.add_trace(go.Scatter(x = category1['X1'], y = category1['X2'], name = '2st category', mode = 'markers', marker=dict(
        color='blue',
    )))
fig.add_trace(go.Scatter(x = df['X1'], y = m*df['X1'] + b, mode = 'lines', name='my classifier'))
fig.add_trace(go.Scatter(x = df['X1'], y = model_m*df['X1'] + model_b, mode = 'lines', name='sklearn classifier'))
fig.update_layout(width= 900, height= 500)

fig.update_yaxes(range=[-2.5, 2.7])

fig.show()

#### Using Sigmoid Function

In [226]:
def step(value):
    return 1 / (1 + np.exp(-value))

In [227]:
class perceptraon:
    def __init__(self, lr, epochs) -> None:
        self.coef_ = None
        self.intercept_ = None
        self.epochs = epochs
        self.learning_rate = lr
    
    def fit(self, X_train, y_train):
        X_train = np.array(X_train)
        y_train = np.array(y_train).ravel()

        X_train = np.insert(X_train, 0, 1, axis = 1)

        beta = np.ones(X_train.shape[1])

        for i in range(self.epochs):
            idx = np.random.randint(0, X_train.shape[0]) # Data Point

            y_hat = step(np.dot(X_train[idx], beta))

            beta = beta + self.learning_rate * (y_train[idx] - y_hat)*X_train[idx]
        
        self.coef_ = beta[1:]
        self.intercept_ = beta[0]

In [228]:
model = perceptraon(0.01, 1000)
model.fit(X_train, y_train)

In [229]:
print(model.coef_, model.intercept_)

[1.4558476  0.93719273] 0.8839371635497871


In [230]:
sigma_m = -model.coef_[0]/model.coef_[1]
sigma_b = -model.intercept_/model.coef_[1]

In [231]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = category0['X1'], y = category0['X2'], name = '1st category', mode = 'markers', marker=dict(
        color='red',
    )))
fig.add_trace(go.Scatter(x = category1['X1'], y = category1['X2'], name = '2st category', mode = 'markers', marker=dict(
        color='blue',
    )))
fig.add_trace(go.Scatter(x = df['X1'], y = m*df['X1'] + b, mode = 'lines', name='my classifier'))
fig.add_trace(go.Scatter(x = df['X1'], y = model_m*df['X1'] + model_b, mode = 'lines', name='sklearn classifier'))
fig.add_trace(go.Scatter(x = df['X1'], y = sigma_m*df['X1'] + sigma_b, mode = 'lines', name='sigmoid classifier'))
fig.update_layout(width= 900, height= 500)

fig.update_yaxes(range=[-2.5, 2.7])

fig.show()

#### Gradient Descent

In [232]:
def sigmoid(value):
    return 1 / (1 + np.exp(-value))

In [233]:
# Define the sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

class GradientDescent:
    def __init__(self, lr=0.01, epochs=1000) -> None:
        self.coef_ = None
        self.intercept_ = None
        self.epochs = epochs
        self.learning_rate = lr
    
    def fit(self, X_train, y_train):
        # Convert inputs to numpy arrays
        X_train = np.array(X_train)
        y_train = np.array(y_train).ravel()  # Ensure y_train is a 1D array
        
        # Add intercept term to X_train
        X_train = np.insert(X_train, 0, 1, axis=1)
        
        # Initialize beta with ones
        beta = np.zeros(X_train.shape[1])  # Better to initialize with zeros

        for epoch in range(self.epochs):
            # Compute the predicted values
            y_hat = sigmoid(np.dot(X_train, beta)) 

            # Compute the gradient
            gradient = np.dot((y_train - y_hat), X_train) / X_train.shape[0]

            # Update beta
            beta += self.learning_rate * gradient
        
        # Extract coefficients and intercept
        self.coef_ = beta[1:]
        self.intercept_ = beta[0]

    def predict(self, X):
        X = np.array(X)
        X = np.insert(X, 0, 1, axis=1)  # Add intercept term
        return sigmoid(np.dot(X, np.concatenate(([self.intercept_], self.coef_))))
    
    def score(self, X, y):
        y_pred = self.predict(X)
        y_pred_class = (y_pred >= 0.5).astype(int)
        return np.mean(y_pred_class == y)

In [234]:
model = GradientDescent(0.001, 10000)
model.fit(X_train, y_train)

In [235]:
print(model.coef_, model.intercept_)

[1.11801297 0.00621458] -0.1555918678717654


In [236]:
gd_m = -model.coef_[0]/model.coef_[1]
gd_b = -model.intercept_/model.coef_[1]

In [239]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = category0['X1'], y = category0['X2'], name = '1st category', mode = 'markers', marker=dict(
        color='red',
    )))
fig.add_trace(go.Scatter(x = category1['X1'], y = category1['X2'], name = '2st category', mode = 'markers', marker=dict(
        color='blue',
    )))
fig.add_trace(go.Scatter(x = df['X1'], y = m*df['X1'] + b, mode = 'lines', name='my classifier'))
fig.add_trace(go.Scatter(x = df['X1'], y = model_m*df['X1'] + model_b, mode = 'lines', name='sklearn classifier'))
fig.add_trace(go.Scatter(x = df['X1'], y = sigma_m*df['X1'] + sigma_b, mode = 'lines', name='sigmoid classifier'))
fig.add_trace(go.Scatter(x = df['X1'], y = gd_m*df['X1'] + gd_b, mode = 'lines', name='gd classifier'))
fig.update_layout(width= 900, height= 500)

fig.update_yaxes(range=[-2.5, 2.7])

fig.show()